In [9]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

import gi
gi.require_version("Gtk", "3.0")
gi.require_version("Gdk", "3.0")
from gi.repository import Gdk, Gtk
gi.require_foreign("cairo")

plt.rcParams["figure.figsize"] = (20, 10)



Wprowadź dalsze przeróbki, dzięki którym odcinki będą zachowywać swoją względną pozycję przy zmianie rozmiarów okna. Kreska, która prawie dotykała prawego brzegu okna, po zmaksymalizowaniu okna dalej powinna być tuż przy jego prawej krawędzi, itp. Najłatwiej to osiągnąć normalizując zapamiętywane współrzędne do postaci liczb zmiennoprzecinkowych z przedziału [0, 1]. W ten sposób punkt w środku kanwy zawsze ma współrzędne (0.5, 0.5), niezależnie od tego jak zmienia się rozmiar okna.

Przy rysowaniu zapamiętanych odcinków oczywiście trzeba ponownie przeliczyć współrzędne. Można to zrobić ręcznie mnożąc je przez aktualną szerokość i wysokość kanwy przed przekazaniem ich do procedur moveto oraz lineto. Alternatywnie można użyć scale i zmienić macierz transformacji układu współrzędnych tak, aby to Cairo wykonywało za nas to mnożenie. Uwaga: to drugie podejście ma niekorzystny efekt uboczny, czy potrafisz powiedzieć jaki?


In [10]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
class Okno(Gtk.Window):
    def __init__(self):
        super(Okno, self).__init__()
        
        self.lines = [Line(None, None)]

        self.set_title("Okno GTK")
        self.set_default_size(400, 300)
        self.connect("destroy", Gtk.main_quit)
        
        da = Gtk.DrawingArea()
        da.set_size_request(200, 200)
        
        da.connect("draw", self.on_draw)
        da.set_events(Gdk.EventMask.BUTTON_PRESS_MASK)
        da.connect("button-press-event", self.on_button_press)
        
        self.add(da)
        self.show_all()

    def on_draw(self, widget, ctx):
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        
        for line in self.lines:
            if line.p1 is not None:
                ctx.arc(line.p1.x * w, line.p1.y * h, 8, 0.0, 6.283)    # ciut mniej niż 2π
                ctx.close_path()
                ctx.stroke()

            if line.p2 is not None:
                ctx.move_to(line.p1.x * w, line.p1.y * h)
                ctx.line_to(line.p2.x * w, line.p2.y * h)
                ctx.stroke()

    def on_button_press(self, widget, event):
        if event.button != 1:
            return False
        
        last_line = self.lines[-1]
        
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        
        if last_line.p1 is None:
            last_line.p1 = Point(event.x / w, event.y / h)
        elif last_line.p2 is None:
            last_line.p2 = Point(event.x / w, event.y / h)
            self.lines.append(Line(None, None))

        widget.queue_draw()

        return True

In [11]:
o = Okno()
Gtk.main()

In [12]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
class Okno(Gtk.Window):
    def __init__(self):
        super(Okno, self).__init__()
        
        self.lines = [Line(None, None)]

        self.set_title("Okno GTK")
        self.set_default_size(400, 300)
        self.connect("destroy", Gtk.main_quit)
        
        da = Gtk.DrawingArea()
        da.set_size_request(200, 200)
        
        da.connect("draw", self.on_draw)
        da.set_events(Gdk.EventMask.BUTTON_PRESS_MASK)
        da.connect("button-press-event", self.on_button_press)
        
        self.add(da)
        self.show_all()

    def on_draw(self, widget, ctx):
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        
        for line in self.lines:
            if line.p1 is not None:
                ctx.arc(line.p1.x * w, line.p1.y * h, 8, 0.0, 6.283)    # ciut mniej niż 2π
                ctx.close_path()
                ctx.stroke()

            if line.p2 is not None:
                ctx.move_to(line.p1.x * w, line.p1.y * h)
                ctx.line_to(line.p2.x * w, line.p2.y * h)
                ctx.stroke()

    def on_button_press(self, widget, event):
        if event.button != 1:
            return False
        
        last_line = self.lines[-1]
        
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        
        if last_line.p1 is None:
            last_line.p1 = Point(event.x / w, event.y / h)
        elif last_line.p2 is None:
            last_line.p2 = Point(event.x / w, event.y / h)
            self.lines.append(Line(None, None))

        widget.queue_draw()

        return True

In [13]:
o = Okno()
Gtk.main()